### 대구 전주 Airbnb reviews

In [1]:
import json
import re

In [2]:
file_lst = ['translated_daegu.json','translated_jeonju.json']

In [3]:
review = {}
for file in file_lst:
    with open(file,'r',encoding='utf-8') as fp:
        review.update(json.load(fp))

In [4]:
review_en={}

for home_id,rev_lsts in review.items():
    review_en[home_id] = []
    for rev_lst in rev_lsts:
        try:
            if str(rev_lst['language'][-2:]) == 'en':
                review_en[home_id].append([rev_lst['rating'],rev_lst['t_comments']])
        except:
            pass

### Data Preprocessing Step
##### 1. Tokenization
##### 2. Stop word elimination
##### 3. Stemming : 단어를 기본형으로 바꾸어준다. 복수형은 단수형으로, 과거형은 현재형으로 바꾸는 과정
##### 4. Representation

In [5]:
import nltk
from nltk import sent_tokenize
from nltk.stem import WordNetLemmatizer
import tqdm

### Tokenization

In [6]:
# 리뷰별 문장단위 tokenizing
for home_id,rev_lst in tqdm.tqdm(review_en.items()) :
    for rev in rev_lst:
        rev[1] = sent_tokenize(rev[1])

100%|█████████████████████████████████████████████████████████████████████████████| 2626/2626 [00:04<00:00, 585.12it/s]


In [7]:
# 길이가 2보다 작은 문장 없애기
for home_id,rev_lst in tqdm.tqdm(review_en.items()) :
    for rev in rev_lst:
        r = []
        for s in rev[1] : 
            if len(s) > 2 :
                r.append(s)
        rev[1] = r        

100%|███████████████████████████████████████████████████████████████████████████| 2626/2626 [00:00<00:00, 38640.09it/s]


###  Stop words elimination

In [8]:
# 소문자와 대문자가 아닌 것은 공백으로 대체
for home_id,rev_lst in review_en.items(): 
    for rev in rev_lst:
        for sentence in rev[1]:
            sentence = re.sub('[^a-zA-Z\s]', '', sentence)

In [9]:
from nltk.corpus import stopwords
stopWords =list(stopwords.words('english'))
print(stopWords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [10]:
add_stopwords = ['airbnb','im','youre','hongdae', 'seoul', 'daegu', 'jeju' ,'busan', 'korea','gangneung','itaewon','myeongdong','john','gangnam','also','even','dont','namsan','incheon','good','great']
# additional_stopwords = []
stopWords = set(stopWords + add_stopwords)

In [11]:
# 문장별 단어단위 tokenizing 후 pos_tag
for home_id,rev_lst in tqdm.tqdm(review_en.items()):
    for rev in rev_lst:
        tmp_lst = []
        for sentence in rev[1]:
            token = nltk.regexp_tokenize(sentence.lower() ,"[\w']+")
            no_stopword_token = [w for w in token if not w in stopWords]
            sent_pos = nltk.pos_tag(no_stopword_token)
            tmp_lst.append(sent_pos)
        rev[1] = tmp_lst

100%|██████████████████████████████████████████████████████████████████████████████| 2626/2626 [15:45<00:00,  2.78it/s]


### 명사, 형용사, 동사, 부사 lemmatization

In [12]:
lemm = WordNetLemmatizer()

def njvr_lemmantizer(sent):
    global lemm
    
    lemm_sent = []
    for word_pos in sent:
        word, pos = word_pos
        if pos[0] == 'N':
            lemm_sent.append(lemm.lemmatize(word,pos='n').lower() +'_N')
        elif pos[0] == 'J':
            lemm_sent.append(lemm.lemmatize(word,pos='a').lower() +'_J')
        elif pos[0] == 'V':
            lemm_sent.append(lemm.lemmatize(word,pos='v').lower() +'_V')
        elif pos[0] == 'R':
            lemm_sent.append(lemm.lemmatize(word,pos='r').lower() +'_R')
        else:pass
    return lemm_sent

In [13]:
for listing_id, rev_lst in tqdm.tqdm(review_en.items()):
    for rev in rev_lst:
        rev_tmp = []
        for sent in rev[1]:
            rev_tmp.append(njvr_lemmantizer(sent))
        rev[1] = rev_tmp

100%|█████████████████████████████████████████████████████████████████████████████| 2626/2626 [00:05<00:00, 497.69it/s]


In [14]:
with open('review_postag_daegujeonju.json','w',encoding='utf-8') as fp:
        json.dump(review_en,fp)

### LDA 

In [15]:
import pandas as pd
import numpy as np
import time

In [16]:
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel

C:\Users\asd00\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [17]:
review = review_en

In [18]:
texts = []
for home_id,rev_lst in review.items():
    for rev in rev_lst:
        for sentence in rev[1] :
            texts.append(sentence)

In [19]:
texts[:2]

[['host_N', 'guide_V', 'well_R', 'check_V', 'check_V', 'problem_N'],
 ['accommodation_N', 'see_V', 'screen_N']]

In [20]:
dic = corpora.Dictionary(texts)

In [21]:
corpus = [dic.doc2bow(text) for text in texts]

In [22]:
len(corpus)

135134

In [23]:
# corpus: the corpus used to train the topic model.
# id2word: a dictionary that maps word numerical identifiers to word strings 
# alpha: the topic distribution. If we set it to 'auto', then we tell the algorithm to determine the topic proportions automatically.
# num_topics: how many topics the algorithm should find.
# passes: how many times the algorithm should go through the corpus. If you are using a large corpus , then this can be set to 1
# model.num_topics: the number of topics.
# model.show_topic(topic_number, topn) returns the probability distribution over words in one topic. for instance [(0.06, 'food'), (0.04, 'cook'), (0.03, 'fry'), ...]
# model.alpha[topic_number]: the prominence of a topic.

#%%time
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 6, id2word = dic, passes=30 ,iterations = 3000,alpha='symmetric', eta='auto') #LDA model 생성

C:\Users\asd00\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [24]:
ldamodel.save('lda_DaeguJunju.lda')

#### Airnb category : 정확성 / 의사소통 / 청결도 / 위치 / 체크인 / 가치

In [25]:
ldamodel.show_topics(num_words=10) #토픽을 구성하는 주요단어들

[(0,
  '0.084*"host_N" + 0.078*"kind_N" + 0.033*"take_V" + 0.021*"make_V" + 0.020*"rest_N" + 0.019*"day_N" + 0.017*"nice_J" + 0.017*"give_V" + 0.017*"thanks_N" + 0.012*"check_V"'),
 (1,
  '0.097*"room_N" + 0.062*"clean_J" + 0.038*"nice_J" + 0.029*"well_R" + 0.022*"comfortable_J" + 0.022*"bathroom_N" + 0.020*"clean_N" + 0.017*"small_J" + 0.017*"really_R" + 0.016*"warm_J"'),
 (2,
  '0.043*"quiet_J" + 0.038*"house_N" + 0.030*"night_N" + 0.028*"little_J" + 0.020*"floor_N" + 0.017*"beautiful_J" + 0.014*"place_N" + 0.013*"hanok_N" + 0.013*"lot_N" + 0.012*"morning_N"'),
 (3,
  '0.063*"accommodation_N" + 0.048*"location_N" + 0.044*"thank_N" + 0.028*"much_J" + 0.021*"price_N" + 0.021*"well_R" + 0.019*"like_V" + 0.018*"enjoy_V" + 0.017*"best_J" + 0.015*"satisfy_V"'),
 (4,
  '0.102*"village_N" + 0.055*"hanok_J" + 0.050*"hanok_N" + 0.028*"locate_V" + 0.025*"location_N" + 0.020*"close_R" + 0.019*"convenient_N" + 0.017*"walk_V" + 0.016*"restaurant_N" + 0.015*"walk_N"'),
 (5,
  '0.061*"go_V" + 0.059*

In [3]:
ldamodel = gensim.models.ldamodel.LdaModel.load('lda_jeju.lda')

In [26]:
import pyLDAvis
import pyLDAvis.gensim 
import matplotlib.pyplot as plt

In [27]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dic)

C:\Users\asd00\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [30]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.226120 -0.132882       1        1  21.417206
2     -0.022532 -0.091161       2        1  17.547485
5      0.092795 -0.062569       3        1  16.878061
4      0.204538  0.321307       4        1  15.975182
0      0.215387 -0.211023       5        1  14.935984
3     -0.264070  0.176329       6        1  13.246089, topic_info=     Category          Freq             Term         Total  loglift  logprob
term                                                                        
84    Default  16435.000000           room_N  16435.000000  30.0000  30.0000
289   Default  12964.000000        village_N  12964.000000  29.0000  29.0000
2     Default   9947.000000           host_N   9947.000000  28.0000  28.0000
107   Default   9206.000000           kind_N   9206.000000  27.0000  27.0000
28    Default  10577.000000          clean_J  10577.000000  26.0000  26.0000
19    Default   7955.000000           time_N   7955.000000  25.0000  25.0000
5     Default   6604.000000  accommodation_N   6604.000000  24.0000  24.0000
14    Default   9124.000000             go_V   9124.000000  23.0000  23.0000
3780  Default   7003.000000          hanok_J   7003.000000  22.0000  22.0000
45    Default   8262.000000       location_N   8262.000000  21.0000  21.0000
1843  Default   8175.000000          hanok_N   8175.000000  20.0000  20.0000
79    Default   5966.000000          quiet_J   5966.000000  19.0000  19.0000
53    Default   5594.000000           stay_N   5594.000000  18.0000  18.0000
16    Default   5364.000000           next_J   5364.000000  17.0000  17.0000
3900  Default   7502.000000         jeonju_N   7502.000000  16.0000  16.0000
32    Default   4631.000000          thank_N   4631.000000  15.0000  15.0000
92    Default   5259.000000           come_V   5259.000000  14.0000  14.0000
44    Default   7831.000000          house_N   7831.000000  13.0000  13.0000
4     Default   7125.000000           well_R   7125.000000  12.0000  12.0000
130   Default   4184.000000          night_N   4184.000000  11.0000  11.0000
54    Default   5483.000000           take_V   5483.000000  10.0000  10.0000
346   Default   3876.000000         little_J   3876.000000   9.0000   9.0000
286   Default   3615.000000         locate_V   3615.000000   8.0000   8.0000
18    Default   5033.000000           stay_V   5033.000000   7.0000   7.0000
310   Default   2958.000000           much_J   2958.000000   6.0000   6.0000
37    Default   3317.000000           want_V   3317.000000   5.0000   5.0000
39    Default   3296.000000          visit_N   3296.000000   4.0000   4.0000
25    Default  10928.000000          place_N  10928.000000   3.0000   3.0000
139   Default   3669.000000       bathroom_N   3669.000000   2.0000   2.0000
90    Default   4458.000000           trip_N   4458.000000   1.0000   1.0000
...       ...           ...              ...           ...      ...      ...
208    Topic6    928.561279       kindness_N    928.761719   2.0213  -4.7306
382    Topic6    867.278992           i've_N    867.481567   2.0212  -4.7988
113    Topic6    831.430054           much_R    831.631470   2.0212  -4.8411
327    Topic6    807.199707  consideration_N    807.403198   2.0212  -4.8706
397    Topic6    751.611389        thought_N    751.811279   2.0212  -4.9420
1190   Topic6    728.595337             tv_N    728.795044   2.0212  -4.9731
73     Topic6    714.474487        careful_J    714.676636   2.0212  -4.9927
273    Topic6    690.339111          relax_V    690.537659   2.0212  -5.0270
673    Topic6    660.386475          thank_R    660.584045   2.0212  -5.0714
415    Topic6    625.748596           full_J    625.946777   2.0212  -5.1253
331    Topic6    609.204346           show_V    609.401306   2.0211  -5.1521
516    Topic6    606.962219    cleanliness_N    607.162292   2.0211  -5.1557
136    Topic6    602.707886      satisfied_J    602.907593  

In [29]:
pyLDAvis.save_html(vis,'LDA_DaeguJunju.html')